In [6]:
import torch
print("GPU Available:", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU Available: True
GPU Name: Tesla T4


In [7]:
import warnings
warnings.filterwarnings("ignore")

!export PYTORCH_CUDA_ALLOC_CONF="expandable_segments:True"
!pip install -q opacus

In [ ]:
import re
import os
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score

from typing import Dict, List, Optional, Tuple, Any, Union, Type

np.random.seed(42)

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(0)

from opacus import PrivacyEngine
from copy import deepcopy
from scipy.stats import mode

# **DATA**

In [ ]:
label_mapping = {
    # DDoS
    'DDoS-ACK_Fragmentation': 'DDoS',
    'DDoS-UDP_Flood': 'DDoS',
    'DDoS-SlowLoris': 'DDoS',
    'DDoS-ICMP_Flood': 'DDoS',
    'DDoS-RSTFINFlood': 'DDoS',
    'DDoS-PSHACK_Flood': 'DDoS',
    'DDoS-HTTP_Flood': 'DDoS',
    'DDoS-UDP_Fragmentation': 'DDoS',
    'DDoS-TCP_Flood': 'DDoS',
    'DDoS-SYN_Flood': 'DDoS',
    'DDoS-SynonymousIP_Flood': 'DDoS',
    'DDoS-ICMP_Fragmentation': 'DDoS',
    
    # DoS
    'DoS-TCP_Flood': 'DoS',
    'DoS-HTTP_Flood': 'DoS',
    'DoS-SYN_Flood': 'DoS',
    'DoS-UDP_Flood': 'DoS',
    
    # Brute Force
    'DictionaryBruteForce': 'Brute Force',
    
    # Spoofing
    'MITM-ArpSpoofing': 'Spoofing',
    'DNS_Spoofing': 'Spoofing',
    
    # Recon
    'Recon-PingSweep': 'Recon',
    'Recon-OSScan': 'Recon',
    'VulnerabilityScan': 'Recon',
    'Recon-PortScan': 'Recon',
    'Recon-HostDiscovery': 'Recon',
    
    # Web-based
    'SqlInjection': 'Web-based',
    'CommandInjection': 'Web-based',
    'Backdoor_Malware': 'Web-based',
    'Uploading_Attack': 'Web-based',
    'XSS': 'Web-based',
    'BrowserHijacking': 'Web-based',
    
    # Mirai
    'Mirai-greip_flood': 'Mirai',
    'Mirai-greeth_flood': 'Mirai',
    'Mirai-udpplain': 'Mirai',
    
    # Benign Traffic
    'BenignTraffic': 'Benign'
}

def GroupAttacks(label):
    return label_mapping.get(label, 'Unknown') # Default to 'unknown'

df_labels = pd.read_csv("..\\datasets\\df_labels.csv")

def preprocessing(df: pd.DataFrame, labels: pd.DataFrame = df_labels) -> pd.DataFrame:
    """ Initial data preprocessing. """
    
    df = df.dropna()
    
    # Column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df['magnitude'] = df['magnitue']
    df = df.drop(['magnitue'], axis=1)

    # Organize label columns
    #     label           2 categories (binary): 0: BenignTraffic, 1: Attack
    #     attack_cat      8 categories: Bening, DDoS, DoS, Recon, Mirai, Web-based, Spoofing, Brute Force
    #     attack_type     34 categories: Benign and 33 attacks
    df['grouped_label'] = df['label'].map(GroupAttacks)
    df.rename(columns={'label': 'attack_type', 'grouped_label': 'attack_cat'}, inplace=True)
    df.replace({'attack_cat': {'BenignTraffic': 'Benign'}, 'attack_type': {'BenignTraffic': 'Benign'}}, inplace=True)
    df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Benign' else 1)

    # Convert categorical labels to numerical labels
    attack_cat_map = dict(zip(labels['attack_cat_name'], labels['attack_cat_num']))
    attack_type_map = dict(zip(labels['attack_type_name'], labels['attack_type_num']))

    df['attack_cat'] = df['attack_cat'].map(attack_cat_map).astype(int)
    df['attack_type'] = df['attack_type'].map(attack_type_map).astype(int)

    # Feature Selection
    less_important_features = ['drate', 'fin_flag_number', 'syn_flag_number', 'rst_flag_number', 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number', 'http', 'dns', 'telnet', 'smtp', 'ssh', 'irc', 'tcp', 'udp', 'dhcp', 'arp', 'icmp', 'ipv', 'llc', 'number']
    df = df.drop(columns=less_important_features)

    return df

In [10]:
scale = StandardScaler() # Initialize a global StandardScaler for reuse across training and test sets

def ScaleData_train(df: pd.DataFrame) -> pd.DataFrame:
    """ Scales training data features using standard normalization (mean=0, std=1). """
    
    df_scaled = scale.fit_transform(df)
    return pd.DataFrame(df_scaled, columns=df.columns) 

def ScaleData_test(df: pd.DataFrame) -> pd.DataFrame:
    """ Scales test/validation data using the training data's fitted scaler """
    
    df_scaled = scale.transform(df)
    return pd.DataFrame(df_scaled, columns=df.columns) 
    

def Split(df: pd.DataFrame, training: bool) -> Tuple[pd.DataFrame, pd.Series, pd.Series]:
    """ Splits the DataFrame into features and labels, and scales features. """
    
    X = df.drop(columns=['label', 'attack_cat', 'attack_type'])
    X_scaled = ScaleData_train(X) if training else ScaleData_test(X)

    # Discard 'atack_cat' as the label will not be used in any model
    y_label = df['label']
    y_attack = df['attack_type']
    
    return X_scaled, y_label, y_attack


def to_tensor(X: Union[pd.DataFrame, torch.Tensor], y: Union[pd.Series, torch.Tensor]) -> TensorDataset:
    """ Converts input features and labels to PyTorch TensorDataset. """
    
    # Convert features to tensor if needed
    if not isinstance(X, torch.Tensor):
        X = torch.tensor(X.to_numpy(), dtype=torch.float32)

    # Convert labels to tensor if needed
    if not isinstance(y, torch.Tensor):
        y = torch.tensor(y.to_numpy(), dtype=torch.long)

    return TensorDataset(X, y)

In [ ]:
NUM_CLIENTS = 3  # Number of federated clients

def LoadingPartitions(PARTITIONS: str) -> Dict[int, Tuple[Tuple[Any, Any, Any], Tuple[Any, Any, Any]]]:
    """ Loads and preprocesses federated data partitions, then splits them into train/validation sets per client, returning processed inputs and labels. """
    
    partitions, train_sets, val_sets, node_data = {}, {}, {}, {}
    
    for i in range(NUM_CLIENTS):
        # Load and preprocess the client's partition
        partitions[i] = preprocessing(pd.read_csv(f"..\\datasets\\{PARTITIONS}\\{(PARTITIONS.replace('/', '_').lower())}_part{i}.csv"))
        
        # Split into Train (70%) and Validation (30%) 
        train_sets[i], val_sets[i] = train_test_split(partitions[i], test_size = 0.3, shuffle=True, random_state=42)

        # Apply custom Split function to get (X, y_bin, y_multi) and store it
        node_data[i] = Split(train_sets[i], True), Split(val_sets[i], False)

    return node_data

In [12]:
def data_loaders(batch: int, MODE: str, node_data: Dict[int, Tuple[Tuple[Any, Any, Any], Tuple[Any, Any, Any]]]) -> Tuple[Dict[int, DataLoader], Dict[int, DataLoader]]:
    """ Creates training and validation PyTorch DataLoaders for each federated client, based on the task mode. """
    
    train_loaders, val_loaders = {}, {}

    for client in range(NUM_CLIENTS):
        (X_train, y_train_bin, y_train_multi), (X_val, y_val_bin, y_val_multi) = node_data[client]

        if MODE == 'AD':
            data_train = X_train
            targets_train = y_train_bin
            data_val = X_val
            targets_val = y_val_bin
        
        else: # MODE == 'AC'
            # Convert to NumPy arrays to avoid indexing issues
            X_train_np = X_train.values if hasattr(X_train, 'values') else X_train
            y_train_multi_np = y_train_multi.values if hasattr(y_train_multi, 'values') else y_train_multi
            X_val_np = X_val.values if hasattr(X_val, 'values') else X_val
            y_val_multi_np = y_val_multi.values if hasattr(y_val_multi, 'values') else y_val_multi

            # Remove benign samples (target == 0) and shift attack labels from 1–33 to 0–32
            mask_train = y_train_multi_np != 0
            data_train = X_train_np[mask_train]
            targets_train = y_train_multi_np[mask_train] - 1 

            mask_val = y_val_multi_np != 0
            data_val = X_val_np[mask_val]
            targets_val = y_val_multi_np[mask_val] - 1
            
        train_loaders[client] = DataLoader(to_tensor(data_train, targets_train), batch_size=batch, shuffle=True)
        val_loaders[client] = DataLoader(to_tensor(data_val, targets_val), batch_size=batch, shuffle=False)

    return train_loaders, val_loaders

# **MODELS**

In [13]:
input_dim = 25

class DNN(nn.Module):
    def __init__(self, input_dim, hidden_layers, hidden_units, activation, output_dim):
        super(DNN, self).__init__()

        layers = []
        in_features = input_dim

        for _ in range(hidden_layers):
            layers.append(nn.Linear(in_features, hidden_units))
            layers.append(activation())  # Activation function
            in_features = hidden_units

        # Output layer
        layers.append(nn.Linear(in_features, output_dim))
        
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


class CNN(nn.Module):
    def __init__(self, input_dim, num_filters, fc_units, dropout, num_conv_layers, output_dim):
        super(CNN, self).__init__()
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=1 if i == 0 else num_filters, out_channels=num_filters, kernel_size=3, padding=1)
            for i in range(num_conv_layers)
        ])
        self.fc = nn.Linear(num_filters * input_dim, fc_units)
        self.output = nn.Linear(fc_units, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        for conv in self.convs:
            x = torch.relu(conv(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(torch.relu(self.fc(x)))
        x = self.output(x)
        return x
        

class CNN_LSTM(nn.Module):
    def __init__(self, input_dim, num_filters, lstm_hidden, num_layers, fc_units, dropout, num_conv_layers, output_dim):
        super(CNN_LSTM, self).__init__()
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=1 if i == 0 else num_filters, out_channels=num_filters, kernel_size=3, padding=1)
            for i in range(num_conv_layers)
        ])
        self.lstm = nn.LSTM(input_size=num_filters, hidden_size=lstm_hidden, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(lstm_hidden, fc_units)
        self.output = nn.Linear(fc_units, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        for conv in self.convs:
            x = torch.relu(conv(x))
        x = x.permute(0, 2, 1)  # Reshape for LSTM (batch, seq_len, features)
        x, _ = self.lstm(x)
        x = self.dropout(torch.relu(self.fc(x[:, -1, :])))
        x = self.output(x)
        return x

# **FEDERATED LEARNING TRAINING**

## **AGGREGATION FUNCTIONS**

In [ ]:
def aggregate_models(model: nn.Module, state_dicts: List[Dict[str, torch.Tensor]], aggregation: str, mu: float = 0.0) -> nn.Module:
    """ Aggregates a list of models using the specified aggregation function. """

    if aggregation == "GeomMedian":
        # Geometric Median aggregation
        return GeomMedian(model, state_dicts)
    
    elif aggregation == "FedAvg":
        # Federated Averaging aggregation: simple averaging of model parameters
        return FedAvg(model, state_dicts)
    
    else:
        raise ValueError(f"Unknown aggregation type: {aggregation}")
      
    
# Geometric Median Aggregation
def GeomMedian(model: nn.Module, state_dicts: List[Dict[str, torch.Tensor]]) -> nn.Module:
    """ Geometric Median aggregation """
    
    keys = state_dicts[0].keys()
    
    # Initialize geometric median with the first model
    median_state_dict = {key: state_dicts[0][key] for key in keys}
    
    # Geometric Median calculation
    for key in keys:
        tensors = torch.stack([state_dict[key].float() for state_dict in state_dicts], dim=0)
        median_state_dict[key] = tensors.median(dim=0)[0] # geometric_median(tensors)
    
    # Create a new model with the aggregated weights
    aggregated_model = model
    aggregated_model.load_state_dict(median_state_dict)
    return aggregated_model


# FedAvg Aggregation
def FedAvg(model: nn.Module, state_dicts: List[Dict[str, torch.Tensor]]) -> nn.Module:
    """ FedAvg Aggregation """
    new_state_dict = state_dicts[0].copy()

    for key in new_state_dict.keys():
        new_state_dict[key] = torch.stack([state_dict[key].float() for state_dict in state_dicts], dim=0).mean(dim=0)
    
    # Create a new model with the aggregated weights
    aggregated_model = model 
    aggregated_model.load_state_dict(new_state_dict)
    return aggregated_model

## **LOCAL TRAINING AND VALIDATION**

In [15]:
def train_local_model(model: nn.Module, global_model: nn.Module, train_loader: DataLoader, val_loader: DataLoader, optimizer: torch.optim.Optimizer, criterion: nn.Module, epochs: int, DP: Optional[Dict[str, float]] = {}, snr: bool = False, mu: float = 0.0) -> Tuple[
        Dict[str, torch.Tensor],  # Trained model weights
        List[float],              # Training accuracies per epoch
        List[float],              # Validation accuracies per epoch
        List[float],              # Balanced validation accuracies per epoch
        List[float],              # Training losses per epoch
        List[float]               # Privacy epsilon values per epoch (if DP)
    ]:
    """ Trains a local model for one federated client, optionally using differential privacy (DP) and tracking SNR for DP. """
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    global_model.to(device)

    train_losses, train_accuracies, val_accuracies, val_balanced_accuracies, DP_epsilon = [], [], [], [], []
    global_weights = {
        name.replace('_module.', ''): param.clone().detach()
        for name, param in global_model.named_parameters()
    }

    if DP:
        privacy_engine = PrivacyEngine()
        model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
            module = model,
            optimizer = optimizer,
            data_loader = train_loader,
            target_epsilon = DP['EPSILON'],
            target_delta = DP['DELTA'],
            epochs = epochs,
            max_grad_norm = DP['MAX_GRAD_NORM'],
            track_clipping=True
        )

    if snr:
        snr_per_epoch, log_snr_per_epoch = [], []
    
    for epoch in range(epochs):
        model.train()
        correct_train, total_train, epoch_loss = 0, 0, 0
        if snr:
            epoch_snr_values = [] # Only needed if SNR tracking is on
        
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            if mu != 0:
                fedprox_term = sum(torch.norm(param - global_weights[name.replace('_module.', '')]) ** 2 
                                   for name, param in model.named_parameters())
                loss += (mu / 2) * fedprox_term
            
            loss.backward()

            if DP and snr:
                # Accumulate per-sample gradient norms
                grad_sample = []
                for p in model.parameters():
                    if hasattr(p, "grad_sample") and p.grad_sample is not None:
                        grad_sample.append(p.grad_sample.view(p.grad_sample.shape[0], -1))  # [batch_size, param_dim]
                
                if grad_sample:
                    stacked = torch.cat(grad_sample, dim=1)  # [batch_size, total_params]
                    batch_mean_grad = stacked.mean(dim=0)    # shape: [total_params]
                    grad_norm_squared = batch_mean_grad.norm(p=2).item() ** 2  # ||ḡ||^2
                    total_params = stacked.shape[1]          # d
                    sigma = DP['MAX_GRAD_NORM'] * optimizer.noise_multiplier  # σ = C × noise multiplier
                    snr_batch = grad_norm_squared / (total_params * sigma ** 2)
                    epoch_snr_values.append(snr_batch)
   
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == y_batch).sum().item()
            total_train += y_batch.size(0)
        
        train_losses.append(epoch_loss / len(train_loader))
        train_accuracies.append(correct_train / total_train)

        # Evaluate on validation set
        model.eval()
        correct_val, total_val = 0, 0
        
        with torch.no_grad():
            all_preds, all_labels = [], []
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                
                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == y_batch).sum().item()
                total_val += y_batch.size(0)

                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(y_batch.cpu().numpy())
        
        val_accuracies.append(correct_val / total_val)
        val_balanced_accuracies.append(balanced_accuracy_score(all_labels, all_preds))
        
        print(f"Epoch {epoch+1}/{epochs} - Loss: {train_losses[-1]:.4f} - Train Acc: {train_accuracies[-1]:.4f} - Val Acc: {val_accuracies[-1]:.4f}  - Bal Val Acc: {val_balanced_accuracies[-1]:.4f}")
        
        if DP:
            eps = privacy_engine.accountant.get_epsilon(DP['DELTA'])
            DP_epsilon.append(eps)
            print(f"         ε = {eps:.2f}, noise: {optimizer.noise_multiplier}")
            
            if snr:
                avg_epoch_snr = sum(epoch_snr_values) / len(epoch_snr_values)
                snr_per_epoch.append(avg_epoch_snr)

                log_snr = math.log10(avg_epoch_snr + 1e-10)  # add epsilon to avoid log(0)
                log_snr_per_epoch.append(log_snr)
                
                print(f"         Avg SNR = {avg_epoch_snr:.4f}")
                print(f"         log10(SNR) = {log_snr:.4f}")

    clean_state_dict = { k.replace('_module.', ''): v for k, v in model.state_dict().items() }
    
    if DP:
        epsilon = privacy_engine.get_epsilon(delta = DP['DELTA'])
        print(f" ------- After the {epochs} local epochs: ε = {epsilon:.2f}")
        if snr:
            print(f"    SNR: {snr_per_epoch} ({log_snr_per_epoch})")
    
    return clean_state_dict, train_accuracies, val_accuracies, val_balanced_accuracies, train_losses, DP_epsilon

In [16]:
def define_optimizer(model: nn.Module, optimizer_name: str, lr: float) -> torch.optim.Optimizer:
    """ Defines and returns an optimizer for the given model based on the optimizer name. """
    
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=lr) 
    return optimizer


def define_activation_fn(activation_name: str) -> Type[nn.Module]:
    """ Returns a PyTorch activation function class based on the given name. """
    
    activation_dict = {
        "ReLU": torch.nn.ReLU,
        "LeakyReLU": torch.nn.LeakyReLU,
        "Tanh": torch.nn.Tanh
    }
    return activation_dict[activation_name]


def divide(input_str: str) -> Tuple[str, float]:
    """ Parses a string formatted as 'Name(value)' and extracts the numerical value. """
    pattern = r"([A-Za-z]+)\s?\(([\d.]+)\)"
    match = re.match(pattern, input_str)
    return "FedAvg", float(match.group(2))


def update_dict(metrics: Dict[str, List], model: str, client: Union[str, int], strategy: str, r: int, train_accuracies: List[float], val_accuracies: List[float],
                val_balanced_accuracies: List[float], train_losses: List[float], epochs: int, DP_epsilon: List[float]) -> Dict[str, List]:
    """ Updates a metrics dictionary with training and evaluation results for a specific client and round. """
    
    metrics['Model'].extend([model]*epochs)
    metrics['Node'].extend([client]*epochs)
    metrics['Strategy'].extend([strategy]*epochs)
    metrics['Round'].extend([r]*epochs)
    metrics['TrainAcc'].extend(train_accuracies)
    metrics['ValAcc'].extend(val_accuracies)
    metrics['BalValAcc'].extend(val_balanced_accuracies)
    metrics['TrainLoss'].extend(train_losses)
    metrics['Epsilon'].extend([(r+1)*eps for eps in DP_epsilon])
    return metrics

## **FULL PROCESS**

In [ ]:
def FederatedLearning(models: Dict[str, torch.nn.Module], model_best_params: Dict[str, Dict[str, Union[str, float, int]]], agg_functions: List[str], 
                      node_data: Dict[int, any], MODE: str, PARTITIONS: str, NUM_ROUNDS: int, EPOCHS: int, DP: Optional[Dict[str, float]] = {}) -> None:
    """ Runs the federated learning training loop across multiple models, strategies, and clients. """
    
    for model in model_best_params.keys():
        print("\n\n============================================================================================================================")

        print(f"\n============== {model} ==============")
        print(f"Parameters: {model_best_params[model]}")
        print(f"Num rounds: {NUM_ROUNDS}\nEpochs per local model: {EPOCHS}")
        
        node_metrics = {'Model': [], 'Node': [], 'Strategy': [], 'Round': [], 'TrainAcc': [], 'ValAcc': [], 'BalValAcc': [], 'TrainLoss': [], 'Epsilon': []}

        train_loaders, val_loaders = data_loaders(2**model_best_params[model]['batch_size'], MODE, node_data)
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        global_model = deepcopy(models[model].to(device))
        global_models = {}
        
        for strategy in agg_functions:
            print(f"\n\n------- {strategy} -------")
            st = strategy
            global_models[strategy] = deepcopy(global_model) # Initiate new global model
            
            for r in range(NUM_ROUNDS):
                print(f"\nRound {r}...")
                
                local_weights = []
                local_model = deepcopy(global_models[strategy]) # Overwrite 'local_model' with 'global_model'
                local_models = {}
                (st, mu) = divide(strategy) if strategy.startswith("FedProx") else (st, 0)
                
                # Train local models 
                for client in range(NUM_CLIENTS):
                    local_model_weights = ' '
                    local_models[client] = deepcopy(local_model) # Identical copy for each node

                    local_model_weights, train_accuracies, val_accuracies, val_balanced_accuracies, train_losses, DP_epsilon = train_local_model(local_models[client], global_models[strategy], train_loaders[client], val_loaders[client], define_optimizer(local_models[client], model_best_params[model]['optimizer'], model_best_params[model]['learning_rate']), nn.CrossEntropyLoss(), EPOCHS, DP, mu)
                    node_metrics = update_dict(node_metrics, model, client, strategy, r, train_accuracies, val_accuracies, val_balanced_accuracies, train_losses, EPOCHS, DP_epsilon)
                    local_weights.append(deepcopy(local_model_weights)) # Store local model weights
    
                # Aggregate local weights to update global model            
                global_models[strategy] = deepcopy(aggregate_models(models[model], local_weights, st, mu))
                # Save global model checkpoint for this round
                path = f"{MODE}\\DP" if DP else MODE
                torch.save(global_models[strategy].state_dict(), f"..\\models\\{path}\\{MODE}{EPOCHS}_{PARTITIONS}_{model}_{strategy}_r{r}.pth")
           
        metrics = pd.DataFrame(node_metrics).round(4)
        modDP = f"-DP{int(DP['EPSILON']*NUM_ROUNDS)}-{DP['MAX_GRAD_NORM']}" if DP else ''
        path = f"{MODE}\\DP" if DP else MODE
        metrics.to_csv(f"..\\metrics\\{path}\\{MODE}{EPOCHS}{modDP}_{PARTITIONS}_Metrics.csv", index=False)

# **ATTACK DETECTION**

In [ ]:
MODE = 'AD' 
CLASSES = 2 if MODE == 'AD' else 33 # 33 types of attacks
NUM_ROUNDS = 10

for m in ['models', 'metrics']:
    os.makedirs(f"..\\{m}\\{MODE}", exist_ok=True)
    os.makedirs(f"..\\{m}\\{MODE}\\DP", exist_ok=True)

AD_model_best_params = {
    'DNN': {'hidden_layers': 3, 'hidden_units': 58, 'activation': 'Tanh', 'learning_rate': 3e-3, 'optimizer': 'Adam', 'batch_size': 9},
    'CNN': {'num_filters': 115, 'fc_units': 60, 'dropout': 0.17, 'num_conv_layers': 3, 'learning_rate': 2e-4, 'optimizer': 'Adam', 'batch_size': 7},
    'CNN-LSTM': {'num_filters': 114, 'lstm_hidden': 64, 'num_layers': 1, 'fc_units': 49, 'dropout': 0.43, 'num_conv_layers': 3, 'learning_rate': 3e-4, 'optimizer': 'Adam', 'batch_size': 5}
}

AD_models = {    
    'DNN': DNN(input_dim, hidden_layers = AD_model_best_params['DNN']['hidden_layers'], hidden_units = AD_model_best_params['DNN']['hidden_units'], activation = define_activation_fn(AD_model_best_params['DNN']['activation']), output_dim=CLASSES),
    'CNN': CNN(input_dim,  AD_model_best_params['CNN']['num_filters'], AD_model_best_params['CNN']['fc_units'],  AD_model_best_params['CNN']['dropout'],  AD_model_best_params['CNN']['num_conv_layers'], output_dim=CLASSES),
    'CNN-LSTM': CNN_LSTM(input_dim, AD_model_best_params['CNN-LSTM']['num_filters'], AD_model_best_params['CNN-LSTM']['lstm_hidden'],  AD_model_best_params['CNN-LSTM']['num_layers'], AD_model_best_params['CNN-LSTM']['fc_units'], AD_model_best_params['CNN-LSTM']['dropout'],  AD_model_best_params['CNN-LSTM']['num_conv_layers'], output_dim=CLASSES)
}

# DP_AD = {'DELTA': 2e-6, 'MAX_GRAD_NORM': 1, 'EPSILON': 2/NUM_ROUNDS}
# DP_AD_HT = {'DELTA': 2e-6, 'MAX_GRAD_NORM': 1.97, 'EPSILON': 2.52/NUM_ROUNDS}
DP_AD, DP_AD_HT = None, None

In [ ]:
for PARTITIONS in ['UNIFORM', 'UNBALANCED']:
    node_data = LoadingPartitions(PARTITIONS)
    
    for EPOCHS in [1, 3, 5, 10]:
        agg_functions = ["GeomMedian", "FedAvg", "FedProx (1)", "FedProx (0.1)", "FedProx (0.01)", "FedProx (0.001)"]
        print(f"Using the {PARTITIONS} partitions.")
        if DP_AD:
            DP_AD = DP_AD_HT if DP_AD_HT else DP_AD
            print(f"Differential Privacy parameters: {DP_AD}")
        FederatedLearning(AD_models, AD_model_best_params, agg_functions, node_data, MODE, PARTITIONS, NUM_ROUNDS, EPOCHS, DP_AD)

# **ATTACK CLASSIFICATION**

In [ ]:
MODE = 'AC'
CLASSES = 2 if MODE == 'AD' else 33 # 33 types of attacks
NUM_ROUNDS = 10

for m in ['models', 'metrics']:
    os.makedirs(f"..\\{m}\\{MODE}", exist_ok=True)
    os.makedirs(f"..\\{m}\\{MODE}\\DP", exist_ok=True)

AC_model_best_params = {
    'DNN': {'hidden_layers': 3, 'hidden_units': 100, 'activation': 'Tanh', 'learning_rate': 2.78e-4, 'optimizer': 'Adam', 'batch_size': 6},
    'CNN': {'num_filters': 51, 'fc_units': 55, 'dropout': 0.1, 'num_conv_layers': 3, 'learning_rate': 1.4e-3, 'optimizer': 'RMSprop', 'batch_size': 7}, # CNN using 25 input features
    'CNN-LSTM': {'num_filters': 71, 'lstm_hidden': 28, 'num_layers': 2, 'fc_units': 61, 'dropout': 0.384, 'num_conv_layers': 1, 'learning_rate': 1.62e-3, 'optimizer': 'RMSprop', 'batch_size': 5},
}

AC_models = {
    'DNN': DNN(input_dim, hidden_layers = AC_model_best_params['DNN']['hidden_layers'], hidden_units = AC_model_best_params['DNN']['hidden_units'], activation = define_activation_fn(AC_model_best_params['DNN']['activation']), output_dim=CLASSES),
    'CNN': CNN(input_dim,  AC_model_best_params['CNN']['num_filters'],  AC_model_best_params['CNN']['fc_units'],  AC_model_best_params['CNN']['dropout'],  AC_model_best_params['CNN']['num_conv_layers'], output_dim=CLASSES),
    'CNN-LSTM': CNN_LSTM(input_dim, AC_model_best_params['CNN-LSTM']['num_filters'], AC_model_best_params['CNN-LSTM']['lstm_hidden'],  AC_model_best_params['CNN-LSTM']['num_layers'], AC_model_best_params['CNN-LSTM']['fc_units'], AC_model_best_params['CNN-LSTM']['dropout'],  AC_model_best_params['CNN-LSTM']['num_conv_layers'], output_dim=CLASSES)
}

# DP_AC = {'DELTA': 2e-6, 'MAX_GRAD_NORM': 1, 'EPSILON': 3/NUM_ROUNDS}
# DP_AC_HT = {'DELTA': 2e-6, 'MAX_GRAD_NORM': 1.99, 'EPSILON':  2.13/NUM_ROUNDS} # MCC: 0.6952
DP_AC, DP_AC_HT = None, None

In [ ]:
for PARTITIONS in ['UNIFORM', 'UNBALANCED']:
    node_data = LoadingPartitions(PARTITIONS)
    
    for EPOCHS in [1, 3, 5, 10]:
        agg_functions = ["GeomMedian", "FedAvg", "FedProx (0.01)", "FedProx (0.001)"]
        print(f"Using the {PARTITIONS} partitions.")
        if DP_AC:
            DP_AC = DP_AC_HT if DP_AC_HT else DP_AC
            print(f"Differential Privacy parameters: {DP_AC}")
        FederatedLearning(AC_models, AC_model_best_params, agg_functions, node_data, MODE, PARTITIONS, NUM_ROUNDS, EPOCHS, DP_AC)